In [1]:
!pip install mlflow boto3 awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 703.4/70

In [3]:
import mlflow
#Setup MLflow tracking server
mlflow.set_tracking_uri("http://18.117.193.162:5000/")

#set experiment
mlflow.set_experiment("Exp 3- TfIdf Trigram max_features")

2025/08/21 16:50:17 INFO mlflow.tracking.fluent: Experiment with name 'Exp 3- TfIdf Trigram max_features' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://campusxalidvcbucket/333842070002886506', creation_time=1755795017303, experiment_id='333842070002886506', last_update_time=1755795017303, lifecycle_stage='active', name='Exp 3- TfIdf Trigram max_features', tags={}>

In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [5]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [8]:
#Step1: Function to run the experiment
def run_experiment_tfidf_max_features(max_features):
  ngram_range = (1,3)
  import seaborn as sns # Import seaborn inside the function

  #Step2: Vectorization using TfIdf with varying max_features
  vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
  X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])
  X_train = vectorizer.fit_transform(X_train)
  X_test = vectorizer.transform(X_test)

  #Step3: Define and train a Random Forest Model
  with mlflow.start_run() as run:
    #Set tags
    mlflow.set_tag("mlflow.runName", f"TfIdf_Trigrams_max_features_{max_features}")
    mlflow.set_tag("experiment_type", "feature_engineering")
    mlflow.set_tag("model_type", "RandomForestClassifier")

    #Add a description
    mlflow.set_tag("description", f"RandomForest with TfIdf Trigrams, max_features = {max_features}")

    #Log vectorizer parameters
    mlflow.log_param("vectorizer_type", "TfIdf")
    mlflow.log_param("ngram_range", ngram_range)
    mlflow.log_param("vectorizer_max_features", max_features)

    #Log Random Forest parameters
    n_estimators = 200
    max_depth = 15

    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)

    #Initialize and trian the model
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    model.fit(X_train, y_train)

    #Step4: Make predictions and log metrics
    y_pred=model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)

    #Log classification report
    classification_rep = classification_report(y_test, y_pred, output_dict= True)
    for label,metrics in classification_rep.items():
      if isinstance(metrics, dict):
        for metric, value in metrics.items():
          mlflow.log_metric(f"{label}_{metric}", value)

    # Log confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix: TF-IDF Trigrams, max_features={max_features}")
    plt.savefig("confusion_matrix.png")
    mlflow.log_artifact("confusion_matrix.png")
    plt.close()

    #Log the model
    mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_{max_features}")

#Step5: Test various max_features values
max_features_values = [1000,2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
  run_experiment_tfidf_max_features(max_features)

2025/08/21 17:08:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/21 17:09:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TfIdf_Trigrams_max_features_1000 at: http://18.117.193.162:5000/#/experiments/333842070002886506/runs/ccb9451c3dd941a89251a7e84bf3b6fb
🧪 View experiment at: http://18.117.193.162:5000/#/experiments/333842070002886506


2025/08/21 17:09:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/21 17:09:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TfIdf_Trigrams_max_features_2000 at: http://18.117.193.162:5000/#/experiments/333842070002886506/runs/7076d68b514c4f6fbe68eb0589f7b4c7
🧪 View experiment at: http://18.117.193.162:5000/#/experiments/333842070002886506


2025/08/21 17:09:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/21 17:09:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TfIdf_Trigrams_max_features_8000 at: http://18.117.193.162:5000/#/experiments/333842070002886506/runs/7b4c0556d80a451da5cece005683819b
🧪 View experiment at: http://18.117.193.162:5000/#/experiments/333842070002886506


2025/08/21 17:14:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/21 17:14:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TfIdf_Trigrams_max_features_9000 at: http://18.117.193.162:5000/#/experiments/333842070002886506/runs/e3498fbb7fee45f0b5628f3e52852531
🧪 View experiment at: http://18.117.193.162:5000/#/experiments/333842070002886506


2025/08/21 17:15:01 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/21 17:15:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TfIdf_Trigrams_max_features_10000 at: http://18.117.193.162:5000/#/experiments/333842070002886506/runs/34934f794b054f569fe01b712319d23c
🧪 View experiment at: http://18.117.193.162:5000/#/experiments/333842070002886506
